In [1]:
my_list = [1,4,7,1,3,5,1,9,6,2]

In [2]:
def the_oracle(my_input):
    winner=9
    if my_input is winner:
        response = True
    else:
        response = False
    return response

In [3]:
for index,trial_number in enumerate(my_list):
    if the_oracle(trial_number) is True:
        print('Winner found at index %i'%index)
        print('%i calls to the Oracle used'%(index+1))

Winner found at index 7
8 calls to the Oracle used


In [4]:
from qiskit import *
import matplotlib.pyplot as plt
import numpy as np

In [6]:
#define the oracle circuit
oracle = QuantumCircuit(2,name='AllSeeingEye')
oracle.cz(0,1)
oracle.to_gate()
oracle.draw()

q_0: ─■─
      │ 
q_1: ─■─

In [8]:
backend = Aer.get_backend('statevector_simulator')
grover_circuit = QuantumCircuit(2,2)
grover_circuit.h([0,1])
grover_circuit.append(oracle,[0,1])
grover_circuit.draw()

┌───┐┌───────────────┐
q_0: ┤ H ├┤0              ├
     ├───┤│  AllSeeingEye │
q_1: ┤ H ├┤1              ├
     └───┘└───────────────┘
c: 2/══════════════════════

In [9]:
job = execute(grover_circuit,backend)
result = job.result()

In [10]:
sv = result.get_statevector()
np.around(sv,2)

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

In [11]:
reflection = QuantumCircuit(2,name='MirrorMirror')
reflection.h([0,1])
reflection.z([0,1])
reflection.cz(0,1)
reflection.h([0,1])
reflection.to_gate()

Instruction(name='MirrorMirror', num_qubits=2, num_clbits=0, params=[])

In [12]:
reflection.draw()

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [13]:
backend = Aer.get_backend('qasm_simulator')
grover_circuit = QuantumCircuit(2,2)
grover_circuit.h([0,1])
grover_circuit.append(oracle,[0,1])
grover_circuit.append(reflection,[0,1])
grover_circuit.measure([0,1],[0,1])

In [14]:
grover_circuit.draw()

┌───┐┌───────────────┐┌───────────────┐┌─┐   
q_0: ┤ H ├┤0              ├┤0              ├┤M├───
     ├───┤│  AllSeeingEye ││  MirrorMirror │└╥┘┌─┐
q_1: ┤ H ├┤1              ├┤1              ├─╫─┤M├
     └───┘└───────────────┘└───────────────┘ ║ └╥┘
c: 2/════════════════════════════════════════╩══╩═
                                             0  1

In [15]:
job=execute(grover_circuit,backend,shots=1)
result=job.result()
result.get_counts()

{'11': 1}